<a href="https://colab.research.google.com/github/KIMHYOENJUN/AI_project/blob/main/txt_parsingipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys 
from google.colab import drive 

drive.mount('./content')

Mounted at ./content


In [2]:
cd content/My Drive/fruit

/content/content/My Drive/fruit


In [3]:
from curses import A_ALTCHARSET
import os
from glob import glob 
import cv2
import sys
import random
import numpy as np
import xml.etree.ElementTree as ET

#os.chdir(r'C:\Users\Administrator\Desktop\test')
#path = r'C:\Users\Administrator\Desktop\test'


class preprocessing(object):
    def __init__(self, resize_size):
      self.resize_size = resize_size

    def resize(self,img):
      resize_img = cv2.resize(img, (self.resize_size, self.resize_size))
      return resize_img

    def normalize(self, img):
      norm_img = img/ 255.0
      return norm_img

    def label_parsing(self, label_path):
      info = ET.parse(label_path)
      root = info.getroot()
      member_object = root.findall('object')
      label_data =[]
      labels_dict = dict(obj_class=[], xmin=[], xmax=[], ymin=[], ymax=[])
      label_data.append(label_path)
      for obj in member_object:
        if obj.find('name').text == 'apple':
          obj_class = 0
        elif obj.find('name').text == 'banana':
          obj_class = 1
        elif obj.find('name').text == 'orange':
          obj_class = 2

      labels_info = obj.find('bndbox')
      xmin = int(labels_info.find('xmin').text) 
      xmax = int(labels_info.find('xmax').text) 
      ymin = int(labels_info.find('ymin').text) 
      ymax = int(labels_info.find('ymax').text) 
      labels_dict['xmin'].append(xmin)
      labels_dict['xmax'].append(xmax)
      labels_dict['ymin'].append(ymin)
      labels_dict['ymax'].append(ymax)
      labels_dict['obj_class'].append(obj_class)
      label_data.append(labels_dict)
      return label_data


def main():
      datapath = '../train_zip/train'
      image_files =glob(os.path.join(datapath,'*.jpg'))
      label_files =glob(os.path.join(datapath,'*.xml'))
      prepro_img = preprocessing(resize_size=244)

      data =list(zip(image_files, label_files))
      random.seed(4)
      random.shuffle(data)


      for file in data:
        image = cv2.imread(file[0])
        label = prepro_img.label_parsing(file[1])


      if(file[0].split('\\'))[-1] == 'mixed_8.jpg':
        rext_img = image

        for i in range(len(label[1]['obj_class'])):
          bb_color =(0, 0, 0)
          if label[1]['obj_class'][i] == 0:
             bb_color =(255, 0, 0)
          elif label[1]['obj_class'][i] == 1: 
            bb_color =(0, 255, 0)
          elif label[1]['obj_class'][i] == 2:
             bb_color =(0, 0, 255)

          rect_img = cv2.rectangle(rect_img,)(label[1]['xmin'][i], label[1]['ymin'][i], label[1]['xmax'][i], label[1]['ymax'][i],label[1]['obj_class'][i])
        cv2.imwrite('bb_result.jpg', rect_img)
      
      with open('train.txt', 'w') as f:
       for file in data:
          label = prepro_img.label_parsing(file[1])
          f.write(label[0]+' ')
          for i in range(len(label[1]['obj_class'])):
             f.write(' {} {} {} {} [] '.format(label[1]['obj_class'][i] ,label[1]['xmin'][i], label[1]['ymin'][i], label[1]['xmax'][i], label[1]['ymax'][i]))
         # f.write(' {} {} {} {} [] '.format(w*[0]['xmin'][i]),int(h*label[0]['ymin'][i]), int(w*label[0]['xmax'][i]), int(h*label[0]['ymax'[i]]), label[0]['obj_class'][i]) 
          f.write('\n')